# Análise de Dados / Solução do problema posto

Com base no modelo de dados construído, vamos buscar solucionar as questões colocadas. Vamos relembrar o problema a ser resolvido e as perguntas propostas:

### Problema a ser resolvido

Sou um consultor de negócios que está apoiando com dados a abertura de uma nova empresa no setor de venda de tintas. É viável abrir esse negócio na cidade de Fundão/ES? Há muitas empresas concorrentes na região? Há fornecedores próximos de onde conseguiria comprar insumos?


### Perguntas a serem respondidas:
- Quantas empresas concorrentes existem de um determinado setor (CNAE) numa determinada localidade (município/UF), onde se deseja abrir a empresa ? São muitas? Se são poucas, pode indicar que não há mercado na região. Se são muitas, pode indicar que o mercado está muito saturado.
- Qual a idade média das empresas que atuam atualmente naquele setor?
- Como estão distribuídas as empresas daquele setor em termos de porte (tamanho da empresa relacionado ao seu faturamento)? Há espaço para empresas menores? As empresas menores são muito recentes ou conseguem sobreviver aos primeiros 3 anos de vida?
- Quantas empresas potencialmente fornecedoras existem nas proximidades daquela localidade (microrregião, mesorregião)?



Então, supondo o contexto abaixo, vamos buscar responder as perguntas acima.

Contexto:
- A empresa que queremos abrir será do setor de venda em varejo (tipo mais comum) de tintas
- A empresa será aberta na cidade de Fundão/ES, cidade pequena do interior do ES, próximo à Serra/ES, Linhares/ES, Aracruz/ES



### 1 - Quantas empresas concorrentes há neste setor (CNAE) e localidade (município/UF) escolhidos?

Para responder a pergunta, primeiro é importante identificar qual o CNAE que estamos tratando. De acordo com o IBGE, o CNAE de venda (em atacado) de tintas é o [**4741500  - Comércio varejista de tintas e materiais para pintura**](https://cnae.ibge.gov.br/?view=subclasse&tipo=cnae&versao=10&subclasse=4741500&chave=tintas)

In [0]:
%sql
select * 
from agg_empresas
where
  CNAE_principal = 4741500 -- Comércio varejista de tintas e materiais para pintura
  and municipio = 'FUNDÃO'
  and UF = 'ES'

municipio,microrregiao_municipio,mesorregiao_municipio,UF,CNAE_principal,desc_CNAE_principal,grupo_CNAE_principal,natureza_juridica,porte,situacao,qtd_empresas,media_idade
FUNDÃO,LINHARES,LITORAL NORTE ESPÍRITO-SANTENSE,ES,4741500,COMÉRCIO VAREJISTA DE TINTAS E MATERIAIS PARA PINTURA,COMÉRCIO VAREJISTA DE MATERIAL DE CONSTRUÇÃO,EMPRESÁRIO (INDIVIDUAL),MICRO,INATIVA,3,null
FUNDÃO,LINHARES,LITORAL NORTE ESPÍRITO-SANTENSE,ES,4741500,COMÉRCIO VAREJISTA DE TINTAS E MATERIAIS PARA PINTURA,COMÉRCIO VAREJISTA DE MATERIAL DE CONSTRUÇÃO,SOCIEDADE EMPRESÁRIA LIMITADA,MICRO,INATIVA,1,null
FUNDÃO,LINHARES,LITORAL NORTE ESPÍRITO-SANTENSE,ES,4741500,COMÉRCIO VAREJISTA DE TINTAS E MATERIAIS PARA PINTURA,COMÉRCIO VAREJISTA DE MATERIAL DE CONSTRUÇÃO,EMPRESÁRIO (INDIVIDUAL),MICRO,ATIVA,1,3.90000


Pelo que vemos acima, no momento desta análise havia somente uma empresa ativa neste CNAE nesta localidade, uma empresa com natureza jurídica Empresário Individual (EI). Houve outras 4 empresas antes, mas todas se encontram inativas atualmente.

Destes resultados, podemos depreender que é um comércio de tintas é um empreendimento relativamente nichado, arriscado mas viável, com mercado consumidor nesta região.

As próximas 2 perguntas vamos responder de forma conjunta:

### 2 - Qual a idade média das empresas que atuam atualmente naquele setor e localidade?



### 3 - Como estão distribuídas as empresas daquele setor em termos de porte? Há espaço para empresas menores? As empresas menores são muito recentes ou conseguem sobreviver aos primeiros 2 anos de vida?

Considerando somente as empresas ativas, vamos recuperar a média de idade das empresas naquele setor e localidade.

In [0]:
%sql

select porte, avg(media_idade) as media_idade, sum(qtd_empresas) as qtd_empresas 
from agg_empresas
where
  CNAE_principal = 4741500 
  and municipio = 'FUNDÃO'
  and UF = 'ES'
  and situacao = 'ATIVA'
group by porte

porte,media_idade,qtd_empresas
MICRO,3.900000000,1


Como havíamos visto, havia apenas uma empresa ativa naquele setor e localidade. Portanto, a média de idade é idêntica à idade daquela empresa em específico, no caso 3,9 anos (ou 3 anos e 11 meses aprox). 

Podemos ver que se trata de uma empresa com porte MICRO (faturamento até 360 mil reais) , empresas essas em geral mais vulneráveis, ainda não tão estabilizadas e com maior taxa de mortalidade - aprox metade das empresas fecham até 3 anos de idade. Entretanto, esta empresa em particular está indo bem e tem mais de 3 anos de idade. Portanto, podemos concluir que há espaço para novos entrantes, para empresas menores e com boa possibilidade de sobrevivência para além da barreira dos 3 anos.

### 4 - Quantas empresas potencialmente fornecedoras existem nas proximidades daquela localidade (microrregião, mesorregião)?

Esta última análise é para verificar se será possível para a futura empresa comprar insumos de forma mais barata, de fornecedores da imediação da localidade em que se encontra. Não haver fornecedores locais para apoiá-la pode ser um grande entrave e encarecer muito o custo de operação daquela empresa, eventualmente inviabilizando o novo negócio.

Para responder essa pergunta, primeiro precisamos entender quais seriam as possíveis empresas fornecedoras desta e buscar na micro e mesorregião.

Como a empresa a ser criada se propõe a vender tintas no varejo, precisamos encontrar as fabricantes de tinta. De acordo com o IBGE, o CNAE correspondente a essa atividade seria o [**2071100 - Fabricação de tintas, vernizes, esmaltes e lacas**](https://cnae.ibge.gov.br/?view=subclasse&tipo=cnae&versao=10.1.0&subclasse=2071100&chave=tintas) . Vamos então buscar as empresas com este CNAE, primeiro no município de Fundão e depois na micro e mesorregião correspondente.

In [0]:
%sql
select * 
from agg_empresas
where
  CNAE_principal = 2071100 -- Fabricação de tintas, vernizes, esmaltes e lacas
  and municipio = 'FUNDÃO'
  and UF = 'ES'
  and situacao = 'ATIVA' -- para esta análise não nos interessam as empresas que não estão ativas

municipio,microrregiao_municipio,mesorregiao_municipio,UF,CNAE_principal,desc_CNAE_principal,grupo_CNAE_principal,natureza_juridica,porte,situacao,qtd_empresas,media_idade


Hummm, mal sinal, não há empresas fornecedoras no município. Mas não percamos as esperanças, vamos olhar a microrregião.

In [0]:
%sql
select * 
from agg_empresas
where
  CNAE_principal = 2071100 -- Fabricação de tintas, vernizes, esmaltes e lacas
  and microrregiao_municipio in (select nome_microrregiao from municipio_bd where upper(nome) = 'FUNDÃO' and upper(sigla_uf) = 'ES')
  and situacao = 'ATIVA' -- para esta análise não nos interessam as empresas que não estão ativas

municipio,microrregiao_municipio,mesorregiao_municipio,UF,CNAE_principal,desc_CNAE_principal,grupo_CNAE_principal,natureza_juridica,porte,situacao,qtd_empresas,media_idade


Ainda nada de empresas fornecedoras nas imediações! Vamos expandir a busca para a mesorregião..

In [0]:
%sql
select * 
from agg_empresas
where
  CNAE_principal = 2071100 -- Fabricação de tintas, vernizes, esmaltes e lacas
  and mesorregiao_municipio in (select nome_mesorregiao from municipio_bd where upper(nome) = 'FUNDÃO' and upper(sigla_uf) = 'ES')
  and situacao = 'ATIVA' -- para esta análise não nos interessam as empresas que não estão ativas

municipio,microrregiao_municipio,mesorregiao_municipio,UF,CNAE_principal,desc_CNAE_principal,grupo_CNAE_principal,natureza_juridica,porte,situacao,qtd_empresas,media_idade


=\ Agora estamos preocupados. Não há empresas fabricantes nas proximidades, nem na localidade, nem na microrregião, nem na mesorregião. Isso poderia indicar que o custo dos insumos poderia ser mais elevado. Mas calma! Não consideramos em nossa análise que empresas de comércio varejista normalmente compram insumos de comércio atacadista. Vejamos:

In [0]:
%sql
select * 
from agg_empresas
where
  CNAE_principal in (
    4679601, -- Comércio atacadista de tintas, vernizes e similares
    4679699 --  Comércio atacadista de materiais de construção em geral 
  ) 
  and mesorregiao_municipio in (select nome_mesorregiao from municipio_bd where upper(nome) = 'FUNDÃO' and upper(sigla_uf) = 'ES')
  and situacao = 'ATIVA' -- para esta análise não nos interessam as empresas que não estão ativas

municipio,microrregiao_municipio,mesorregiao_municipio,UF,CNAE_principal,desc_CNAE_principal,grupo_CNAE_principal,natureza_juridica,porte,situacao,qtd_empresas,media_idade


Também não temos nada de comércio atacadista nas imediações... =\
 


Porém, um leitor atento verá que consideramos apenas as empresa cuja atividade *principal* é de fabricação ou comércio atacadista de tintas. Na realidade, empresas podem desempenhar essas atividades, mas não ser a sua atividade principal, de forma que uma pesquisa pelo `CNAE_principal` pode não ser adequada. Para uma busca completa, vamos recorrer à base granular:


In [0]:
%sql
select * 
from bh_empresas
where
  (
    CNAEs_secundarios like '%2071100%' -- que contenha na lista de CNAEs secundários o CNAE de Fabricação de tintas, vernizes, esmaltes e lacas, em qq ordem
    or CNAEs_secundarios like '%4679601%' -- que contenha na lista de CNAEs secundários o CNAE de Comércio atacadista de tintas, vernizes e similares, em qq ordem
    or CNAEs_secundarios like '%4679699%' -- que contenha na lista de CNAEs secundários o CNAE de Comércio atacadista de materiais de construção em geral, em qq ordem
  ) 
  and mesorregiao_municipio in (select nome_mesorregiao from municipio_bd where upper(nome) = 'FUNDÃO' and upper(sigla_uf) = 'ES')
  and situacao = 'ATIVA' -- para esta análise não nos interessam as empresas que não estão ativas

cnpj_raiz,cnpj_completo,nome,natureza_juridica,porte,CNAE_principal,desc_CNAE_principal,grupo_CNAE_principal,CNAEs_secundarios,municipio,microrregiao_municipio,mesorregiao_municipio,UF,situacao,idade_atual


Bom, agora realmente esgotamos possibilidades na microrregião e mesorregião imediatas de Fundão/ES. Vamos considerar como último recurso de viabilidade do empreendimento, a compra de qq município do ES: 

In [0]:
%sql
select * 
from bh_empresas
where
  (
    CNAEs_secundarios like '%2071100%' -- que contenha na lista de CNAEs secundários o CNAE de Fabricação de tintas, vernizes, esmaltes e lacas, em qq ordem
    or CNAEs_secundarios like '%4679601%' -- que contenha na lista de CNAEs secundários o CNAE de Comércio atacadista de tintas, vernizes e similares, em qq ordem
    or CNAEs_secundarios like '%4679699%' -- que contenha na lista de CNAEs secundários o CNAE de Comércio atacadista de materiais de construção em geral, em qq ordem
  ) 
  and UF = 'ES'
  and situacao = 'ATIVA' -- para esta análise não nos interessam as empresas que não estão ativas

cnpj_raiz,cnpj_completo,nome,natureza_juridica,porte,CNAE_principal,desc_CNAE_principal,grupo_CNAE_principal,CNAEs_secundarios,municipio,microrregiao_municipio,mesorregiao_municipio,UF,situacao,idade_atual
00005275,00005275000460,KAEFER TECKMA ENGENHARIA LTDA,SOCIEDADE EMPRESÁRIA LIMITADA,DEMAIS,3321000,INSTALAÇÃO DE MÁQUINAS E EQUIPAMENTOS INDUSTRIAIS,INSTALAÇÃO DE MÁQUINAS E EQUIPAMENTOS,"2731700,4120400,4292802,4321500,4679699,7112000",SERRA,VITÓRIA,CENTRAL ESPÍRITO-SANTENSE,ES,ATIVA,12.5
05057914,05057914000455,PARENTE ANDRADE LTDA,SOCIEDADE EMPRESÁRIA LIMITADA,DEMAIS,8121400,LIMPEZA EM PRÉDIOS E EM DOMICÍLIOS,ATIVIDADES DE LIMPEZA,"0161001,0161002,0161003,0230600,2052500,3311200,3314704,3314707,3701100,3811400,3812200,3821100,3822000,4120400,4313400,4322302,4391600,4399101,4399104,4679699,4693100,4923002,4929901,4929902,4930203,5030101,5030102,5030103,5590603,5590699,5620101,7111100,7112000,7119701,7119702,7119703,7119799,7711000,7732201,7732202,7810800,7830200,8130300,9601701,9601703",SERRA,VITÓRIA,CENTRAL ESPÍRITO-SANTENSE,ES,ATIVA,5.9
07955224,07955224000104,LAMINAR LAMINADOS DO BRASIL,SOCIEDADE ANÔNIMA FECHADA,DEMAIS,2599399,FABRICAÇÃO DE OUTROS PRODUTOS DE METAL NÃO ESPECIFICADOS ANTERIORMENTE,FABRICAÇÃO DE PRODUTOS DE METAL NÃO ESPECIFICADOS ANTERIORMENTE,"4685100,4669901,4679699",CACHOEIRO DE ITAPEMIRIM,CACHOEIRO DE ITAPEMIRIM,SUL ESPÍRITO-SANTENSE,ES,ATIVA,18.9
11063779,11063779000100,DIMENSAO COMERCIO DE ROCHAS ORNAMENTAIS LTDA,SOCIEDADE EMPRESÁRIA LIMITADA,PEQUENA,4679602,COMÉRCIO ATACADISTA DE MÁRMORES E GRANITOS,"COMÉRCIO ATACADISTA DE MADEIRA, FERRAGENS, FERRAMENTAS, MATERIAL ELÉTRICO E MATERIAL DE CONSTRUÇÃO",4679699,CACHOEIRO DE ITAPEMIRIM,CACHOEIRO DE ITAPEMIRIM,SUL ESPÍRITO-SANTENSE,ES,ATIVA,15.6
14559760,14559760000148,AMV DISTRIBUIDORA,SOCIEDADE EMPRESÁRIA LIMITADA,DEMAIS,4651601,COMÉRCIO ATACADISTA DE EQUIPAMENTOS DE INFORMÁTICA,COMÉRCIO ATACADISTA DE EQUIPAMENTOS E PRODUTOS DE TECNOLOGIAS DE INFORMAÇÃO E COMUNICAÇÃO,"2621300,2622100,2824101,3101200,3102100,3312103,3314707,3329501,4635401,4635402,4635499,4639701,4641902,4641903,4642701,4643501,4643502,4646001,4647801,4649401,4649404,4649405,4649406,4649499,4651602,4652400,4669999,4673700,4679699,4691500,4753900,4759899,4762800,4773300,4930201,4930202,6201501,6203100,6209100,6810202,7711000,7719599,7732201,7732202,7733100,7739002,7739099,9511800,9512600,9529105",CARIACICA,VITÓRIA,CENTRAL ESPÍRITO-SANTENSE,ES,ATIVA,13.4
15801747,15801747000205,SEVEN SUPRIMENTOS INDUSTRIAIS,SOCIEDADE EMPRESÁRIA LIMITADA,DEMAIS,4652400,COMÉRCIO ATACADISTA DE COMPONENTES ELETRÔNICOS E EQUIPAMENTOS DE TELEFONIA E COMUNICAÇÃO,COMÉRCIO ATACADISTA DE EQUIPAMENTOS E PRODUTOS DE TECNOLOGIAS DE INFORMAÇÃO E COMUNICAÇÃO,"4530701,4613300,4623109,4623199,4632001,4642702,4644302,4649402,4651601,4663000,4672900,4673700,4679699,4692300,7319002",SERRA,VITÓRIA,CENTRAL ESPÍRITO-SANTENSE,ES,ATIVA,5.1
17315257,17315257000135,B & F DISTRIBUIDORA,SOCIEDADE EMPRESÁRIA LIMITADA,DEMAIS,4639701,COMÉRCIO ATACADISTA DE PRODUTOS ALIMENTÍCIOS EM GERAL,"COMÉRCIO ATACADISTA ESPECIALIZADO EM PRODUTOS ALIMENTÍCIOS, BEBIDAS E FUMO","4632003,4633801,4633802,4635402,4635499,4637102,4637103,4637105,4637107,4641903,4643501,4646001,4646002,4647801,4649408,4649409,4672900,4673700,4679601,4679699",IBATIBA,ALEGRE,SUL ESPÍRITO-SANTENSE,ES,ATIVA,12.3
24162678,24162678000139,DELLA PIETRA,SOCIEDADE EMPRESÁRIA LIMITADA,PEQUENA,4679602,COMÉRCIO ATACADISTA DE MÁRMORES E GRANITOS,"COMÉRCIO ATACADISTA DE MADEIRA, FERRAGENS, FERRAMENTAS, MATERIAL ELÉTRICO E MATERIAL DE CONSTRUÇÃO","4110700,4662100,4663000,4672900,4673700,4679699,4681805,4684201,5211799,7020400,7490104,8219999",CACHOEIRO DE ITAPEMIRIM,CACHOEIRO DE ITAPEMIRIM,SUL ESPÍRITO-SANTENSE,ES,ATIVA,9.1
27995686,27995686000235,MARC,SOCIEDADE EMPRESÁRIA LIMITADA,MICRO,4651601,COMÉRCIO ATACADISTA DE EQUIPAMENTOS DE INFORMÁTICA,COMÉRCIO ATACADISTA DE EQUIPAMENTOS E PRODUTOS DE TECNOLOGIAS DE INFORMAÇÃO E COMUNICAÇÃO,"3329501,4321500,4647801,4649499,4673700,4679699,6622300

Ótimo, temos muitas opções no ES, algumas até relativamente próximas, no município de Serra/ES, Vila Velha/ES, Vitória/ES, Cariacica/ES e Linhares/ES.



### Conclusões Finais

Com a ajuda das bases de dados criadas, com relativo pouco esforço de análise, pudemos comprovar que é viável abrir um empreendimento de venda de tintas na localidade de Fundão/ES . Entretanto, chama a atenção que não há muitas empresas desse tema na região, que não fornecedores nas proximidades imediatas. Isso denota que o empreendimento carrega um maior risco e que deve ser aprofundada a análise com outros elementos e fatores, como custo de insumos, preços, margem de lucro, necessidade de aporte de capital, etc, todos fora do escopo deste trabalho, para fundamentar finalmente a decisão ou não de abrir este empreendimento.